In [1]:
from utils import *
from sknn_sparse_matrix_numba import *
import pandas as pd
import numpy as np
import time
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import norm
from tqdm import tqdm

In [2]:
FOLDER_TRAIN = 'data/train/'
FOLDER_TEST = 'data/test/'
train, test = load_dataset(FOLDER_TRAIN, FOLDER_TEST, feather=True)

load_dataset
 -- loaded in: 0.8099935054779053s
 -- filtered in: 1.3749723434448242s
 -- actions:  6677800
 -- items:  681805
 -- lists:  100000


In [3]:
df = train['actions'][['playlist_id', 'track_id']]
df['track_id'] -= 1
df['rating'] = np.ones(df.shape[0])
df = df.rename(columns={"playlist_id": "user", "track_id": "item"})

C:\Users\nanic\AppData\Local\Temp\ipykernel_13272\493546403.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['track_id'] -= 1
C:\Users\nanic\AppData\Local\Temp\ipykernel_13272\493546403.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rating'] = np.ones(df.shape[0])


In [4]:
rec = UserUser(2000, feedback='implicit')

In [5]:
start = time.time()
rec = rec.fit(df)
end = time.time()
print('Training Time: {}'.format(end - start))

Mapping build time:  1.0700881481170654
Training Time: 2.3020172119140625


In [6]:
n = []
s = []
p = []
pids = np.random.choice(train['playlists']['playlist_id'].values, 10)
for pid in tqdm(pids):
    start = time.time()
    _, n_time, s_time = rec.predict_for_user(pid, np.unique(df['item'].values))
    end = time.time()
    p.append(end - start)
    n.append(n_time)
    s.append(s_time)
print('Mean find neighbor time: ', np.mean(n))
print('Mean score item time: ', np.mean(s))
print('Mean prediction time: ', np.mean(p))

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:10<00:00,  7.06s/it]

Mean find neighbor time:  0.046883654594421384
Mean score item time:  6.62051248550415
Mean prediction time:  7.060955619812011
